This file contains a notebook file called “Final_Exam_Starter_Code.ipynb” which has
some initial codes and helper functions for the final exam.
1. Load the data file “final_exam_text_data.csv” which contains 672 rows with two
columns about consumer complaint narrative and the product it is related to. The products
have 5 different categories including ‘Debt Collection’, ‘Consumer Loan’, ‘Mortgage’,
‘Credit Card’ and ‘Student Loan’.

In [ ]:
import pandas as pd
import numpy as np
from numpy import hstack
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, LSTM, GRU, Embedding
import matplotlib.pyplot as plt
from scipy.ndimage.interpolation import shift
from tensorflow.keras.utils  import to_categorical
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SpatialDropout1D, LSTM, GRU, Dense, Activation, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Flatten

<ipython-input-102-eb614f11b78d>:6: DeprecationWarning: Please import `shift` from the `scipy.ndimage` namespace; the `scipy.ndimage.interpolation` namespace is deprecated and will be removed in SciPy 2.0.0.
  from scipy.ndimage.interpolation import shift


In [ ]:
def rmse(y_pred, y_real):
    return np.sqrt(np.sum((y_pred - y_real)**2)  / len(y_real))

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving final_exam_text_data.csv to final_exam_text_data.csv


In [ ]:
df_text =  uploaded

In [ ]:
df_text= pd.read_csv("final_exam_text_data.csv", sep = ',')

In [ ]:
df_text.head()

,consumer_complaint_narrative,product
0,XXXX has claimed I owe them {$27.00} for XXXX ...,Debt collection
1,Due to inconsistencies in the amount owed that...,Consumer Loan
2,In XX/XX/XXXX my wages that I earned at my job...,Mortgage
3,I have an open and current mortgage with Chase...,Mortgage
4,XXXX was submitted XX/XX/XXXX. At the time I s...,Mortgage


In [ ]:
df_text.shape

(672, 2)

In [ ]:
df_text['product'].unique()

array(['Debt collection', 'Consumer Loan', 'Mortgage', 'Credit card',
       'Student loan'], dtype=object)

In [ ]:
df_text['consumer_complaint_narrative'].unique()

array(['XXXX has claimed I owe them {$27.00} for XXXX years despite the PROOF of PAYMENT I sent them : canceled check and their ownPAID INVOICE for {$27.00}! \nThey continue to insist I owe them and collection agencies are after me. \nHow can I stop this harassment for a bill I already paid four years ago? \n',
       'Due to inconsistencies in the amount owed that I was told by M & T Bank and the amount that was reported to the credit reporting agencies, I was advised to write a good will letter in order to address the issue and request the negative entry be removed from my credit report all together. I had a vehicle that was stolen and it was declared a total loss by insurance company. The insurance company and the GAP insurancw companypaid the outstanding balance of the loan, but I was told by M & T Bank that there was still a balance due on the loan. In good faith, without having received any proof as to why there was still a balance, I made a partial payment towards the remaining 

In [ ]:
df_text.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 672 entries, 0 to 671
Data columns (total 2 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   consumer_complaint_narrative  672 non-null    object
 1   product                       672 non-null    object
dtypes: object(2)
memory usage: 10.6+ KB


In [ ]:
df_text.describe()

,consumer_complaint_narrative,product
count,672,672
unique,666,5
top,This company continues to report on my credit ...,Debt collection
freq,3,255


In [ ]:
df_text.isnull().sum()

consumer_complaint_narrative    0
product                         0
dtype: int64

a. Tokenize the data using max features of size 10000 and pad the sequences to
make length of 100. Finally, divide the data into 80/20 train test splits.


In [ ]:
max_features = 10000
max_len = 100

text = df_text['consumer_complaint_narrative']
labels = df_text['product']
# Creates a tokenizer, configured to only take
#into account the 1,000 most common words
tokenizer = Tokenizer(num_words = 1000)

# builds the word index
tokenizer.fit_on_texts(text)
# recover the word index that was computed
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
print('unique tokens: %s' %(word_index))


Found 6386 unique tokens.
unique tokens: {'xxxx': 1, 'the': 2, 'i': 3, 'to': 4, 'and': 5, 'a': 6, 'my': 7, 'of': 8, 'that': 9, 'was': 10, 'they': 11, 'in': 12, 'on': 13, 'have': 14, 'for': 15, 'this': 16, 'me': 17, 'not': 18, 'is': 19, 'with': 20, 'it': 21, 'xx': 22, 'from': 23, 'credit': 24, 'we': 25, 'had': 26, 'as': 27, 'loan': 28, 'account': 29, 'payment': 30, 'be': 31, 'would': 32, 'them': 33, '00': 34, 'by': 35, 'at': 36, 'been': 37, 'or': 38, 'but': 39, 'told': 40, 'an': 41, 'no': 42, 'mortgage': 43, 'do': 44, 'did': 45, 'are': 46, 'has': 47, 'debt': 48, 'bank': 49, 'were': 50, 'received': 51, 'when': 52, 'all': 53, "n't": 54, 'company': 55, 'which': 56, 'am': 57, 'payments': 58, 'their': 59, 'after': 60, 'time': 61, 'information': 62, 'so': 63, 'called': 64, 'pay': 65, 'our': 66, 'sent': 67, "''": 68, 'any': 69, 'she': 70, 'call': 71, 'paid': 72, 'if': 73, 'letter': 74, 'because': 75, 'get': 76, 'he': 77, 'never': 78, 'will': 79, 'phone': 80, "'s": 81, 'up': 82, 'out': 83, 'the

In [ ]:
sequences = tokenizer.texts_to_sequences(text)
print(sequences)
# one-hot binary representations. vectorization models other than one-hot encoding are supported by this vectorized
one_hot_results = tokenizer.texts_to_matrix(text, mode='binary')
print(one_hot_results.shape)
print('one_hot_results: %s\n %s' %(one_hot_results[0], one_hot_results[1]))

[[1, 47, 824, 3, 243, 33, 34, 15, 1, 101, 447, 2, 274, 8, 30, 3, 67, 33, 138, 5, 59, 15, 34, 11, 295, 4, 3, 243, 33, 5, 110, 669, 46, 60, 17, 158, 91, 3, 268, 16, 719, 15, 6, 156, 3, 254, 72, 101, 325], [89, 4, 12, 2, 99, 288, 9, 3, 10, 40, 35, 49, 5, 2, 99, 9, 10, 422, 4, 2, 24, 192, 669, 3, 10, 335, 4, 670, 6, 307, 79, 74, 12, 352, 4, 266, 2, 164, 5, 139, 2, 530, 31, 303, 23, 7, 24, 104, 53, 987, 3, 26, 6, 344, 9, 10, 5, 21, 10, 6, 474, 493, 35, 185, 55, 2, 185, 55, 5, 2, 988, 2, 720, 114, 8, 2, 28, 39, 3, 10, 40, 35, 49, 9, 88, 10, 109, 6, 114, 89, 13, 2, 28, 12, 307, 863, 211, 304, 51, 69, 274, 27, 4, 176, 88, 10, 109, 6, 114, 3, 100, 6, 30, 740, 2, 741, 48, 3, 84, 67, 2, 74, 109, 4, 65, 2, 8, 2, 48, 39, 12, 15, 2, 8, 2, 530, 13, 7, 24, 104, 36, 132, 312, 12, 1, 165, 379, 7, 24, 140, 21, 952, 6, 572, 114, 8, 908, 34, 39, 52, 3, 796, 7, 24, 104, 112, 13, 1, 1, 165, 88, 10, 6, 572, 114, 8, 34, 2, 23, 2, 988, 185, 5, 7, 185, 55, 47, 37, 72, 49, 573, 9, 3, 109, 243, 34, 8, 56, 34, 47, 

In [166]:
X = one_hot_results
y= labels

In [167]:
y.shape

(672,)

In [169]:
X.shape

(672, 1000)

In [170]:
y = to_categorical(labels, num_classes=num_classes)

In [ ]:
embedding_layer = Embedding(1000, 64) # args: # of possible tokens, and dim of embeddings
embedding_layer

<Embedding name=embedding_22, built=False>

In [ ]:
X= df_text.iloc[:, :-1]
y= df_text.iloc[:, -1]

In [ ]:
max_features = 10000 # Number of words to consider as features

# Loads the data as lists of integers
(X_train, y_train, X_test, y_test) = train_test_split(one_hot_results, y, random_state= 42, stratify= None)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(504, 1000) (168, 1000)
(504,) (168,)


b. Train a 1D CNN model for group prediction task. Use above max_features and
max_len values along with 128 dimensional embeddings in Embedding
layer.Model architecture should contain following layers:
Conv1D(32, 7, activation='relu')
MaxPooling1D(5)
Conv1D(32, 7, activation='relu')
GlobalMaxPooling1D()


In [ ]:

# Tokenize and pad sequences
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(text)
sequences = tokenizer.texts_to_sequences(text)
X2 = pad_sequences(sequences, maxlen=max_len)

# Convert labels to numpy array
y = np.array(labels)

In [171]:
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df_text['product'] = label_encoder.fit_transform(df_text['product'])

# Convert labels to one-hot encoding
num_classes = len(np.unique(labels))  # Number of classes


In [ ]:
text = df_text['consumer_complaint_narrative'].values
labels = df_text['product'].values

Set your optimizer to RMSprop(lr=1e-4), loss to categorical_crossentropy and metrics to
accuracy. Train for 200 epochs with batch_size 128 and use a validation_split of 0.2 .


In [172]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dense


max_features = 10000
max_len = 100
embedding_dim= 128
num_classes = 5
# Define the model
model = Sequential([
    Embedding(input_dim=max_features, output_dim=128, input_length=max_len),
    Conv1D(32, 7, activation='relu'),
    MaxPooling1D(5),
    Conv1D(32, 7, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(num_classes, activation='softmax')  # num_classes should be the number of unique labels
])

# Compile the model
optimizer = RMSprop(learning_rate=1e-4)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [159]:
X.shape

(672, 1)

In [160]:
y.shape

(672,)

In [175]:
# Train the model
history = model.fit(X_train, y_train epochs=200,batch_size=128,validation_split=0.2)

Epoch 1/200


AttributeError: 'NoneType' object has no attribute 'items'

In [ ]:
print("X_train shape:", X_train.shape)  # Should be (num_samples, max_len)
print("y_train shape:", y_train.shape)  # Should be (num_samples, num_classes)

X_train shape: (504, 1000)
y_train shape: (168, 1000)


In [164]:
X_train.shape

(537, 100)

In [165]:
y_train.shape

(537,)

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

i. Generate the validation loss/accuracy plots and select the best number of
epochs.


In [ ]:
#loss
history_dict = history.history

loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']

epochs = range(1, len(loss_values) + 1)

plt.plot(epochs, loss_values, 'bo', label='Training loss')
plt.plot(epochs, val_loss_values, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
#accuracy
history_dict = history.history

acc_values = history_dict['acc']
val_acc_values = history_dict['val_acc']

epochs = range(1, len(acc_values) + 1)

plt.plot(epochs, acc_values, 'bo', label='Training acc')
plt.plot(epochs, val_acc_values, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

ii. Calculate the accuracy of the model over the test set by retraining the
model with the best epoch number from part i.

In [ ]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy:.4f}")

c. Add dropout regularization with a rate of 0.2 right after the MaxPooling1D(5)
layer and train the model

In [ ]:
# Define the model with dropout
model = Sequential([
    Embedding(input_dim=max_features, output_dim=128, input_length=max_len),
    Conv1D(32, 7, activation='relu'),
    MaxPooling1D(5),
    Conv1D(32, 7, activation='relu'),
    GlobalMaxPooling1D(),
    Dropout(0.2),  # Dropout layer added after GlobalMaxPooling1D
    Dense(64, activation='relu'),
    Dropout(0.2),  # Dropout layer added before the output layer
    Dense(num_classes, activation='softmax')  # Output layer for multi-class classification
])

# Compile the model
optimizer = RMSprop(learning_rate=1e-4)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history2 = model.fit(X_train, y_train epochs=200,batch_size=128,validation_split=0.2)

i. Select the best number of epochs and calculate the accuracy of the
model over the test set by retraining the model with the best epoch
number.

In [ ]:
best_epoch_model = np.argmin(history.history['val_loss'])
print("best epoch:%s validation loss:%.3f validation acc:%.3f" %(
best_epoch_model+1,
history.history['val_loss'][best_epoch_model],
history.history['val_accuracy'][best_epoch_model]))

In [ ]:
#best epoch
history3 = model.fit(X_train, y_train epochs=200,batch_size=128,validation_split=0.2)

ii. Compare the accuracy of models from part b and c and observe if
regularization affected the performance of the model.

In [ ]:
#Evaluate the best model on the test set
test_loss, test_acc = model.evaluate(X_train, y_train)
print(f"Test accuracy for Model: {test_acc}")
print(f"Test loss for Model: {test_loss}")
# Evaluate the best model on the test set
test_loss, test_acc = model.evaluate(X_train, y_train)
print(f"Test accuracy for Dropout Model: {test_acc}")
print(f"Test loss for Dropout Model: {test_loss}")

2. Data file “final_exam_TS_data.csv” contains 144 rows of one column called Quantity
over a specific range of time declared under the column “Month”.
Load the data using the “Month” column as Pandas data frame index.Run the following
code to set the convert month column into monthly observations.

 (The code is already
given).
df = pd.read_csv('final_exam_TS_data.csv')
df = df.rename(columns={'Month':'Date'})
df['month'] = pd.DatetimeIndex(df['Date']).month
df


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving final_exam_TS_data.csv to final_exam_TS_data.csv


In [ ]:
df=  uploaded

In [ ]:
df = pd.read_csv('final_exam_TS_data.csv')


In [ ]:
df = df.rename(columns={'Month':'Date'})
df['month'] = pd.DatetimeIndex(df['Date']).month

In [ ]:
df

,Date,Quantity,month
0,1949-01,112,1
1,1949-02,118,2
2,1949-03,132,3
3,1949-04,129,4
4,1949-05,121,5
...,...,...,...
139,1960-08,606,8
140,1960-09,508,9
141,1960-10,461,10
142,1960-11,390,11


a. The function for splitting the data into input and output sequences as well as
train and test split is given in the file “Final_Exam_Starter_Code.ipynb” and is
called “split_series”. Using the function for prediction_horizon of 12 and
look_back value of 24. Train a stacked GRU with the following specifications:
2 GRU layers with 50 and 25 hidden units and ‘swish’ activation function,
rmsprop optimizer with learning_rate 0.001, and loss function of mse, batch size
128 and number of epochs 200.

In [177]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.losses import MeanSquaredError

In [180]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps_in
		out_end_ix = end_ix + n_steps_out-1
		# check if we are beyond the dataset
		if out_end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1:out_end_ix, -1]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)

In [182]:
prediction_horizon = 12
look_back = 24
n_features = 1 # we only have the time series as the input

In [183]:
# split a univariate dataset into train/test sets
def split_dataset(data, prediction_horizon, look_back):
    # split into standard days (e..g., last (48=prediction_horizon + look_back) hours are test input and label)
    train, test = data[:-(prediction_horizon)], data[-(prediction_horizon + look_back):]
    print("before split")
    print(train.shape)
    print(test.shape)
    print('')

    # restructure into windows of weekly data
    train = np.array(np.split(train, len(train)/24))
    test = np.array(np.split(test, len(test)/24))

    return train, test

In [187]:
# convert history into inputs and outputs
def to_supervised(train, look_back, n_out=7):

    # flatten data
    data = train.reshape((train.shape[0]*train.shape[1], train.shape[2]))

    X, y = list(), list()

    in_start = 0
    # step over the entire history one time step at a time
    for _ in range(len(data)):
        # define the end of the input sequence
        in_end = in_start + look_back
        out_end = in_end + n_out

        # ensure we have enough data for this instance
        if out_end <= len(data):
            x_input = data[in_start:in_end, 0]
            x_input = x_input.reshape((len(x_input), 1))
            X.append(x_input)
            y.append(data[in_end:out_end, 0])

        # move along one time step
        in_start += 1

    return np.array(X), np.array(y)

In [188]:
# look_back = n_steps
look_back = 24

X_data = df["Date"].values
X_data = X_data.reshape((X_data.shape[0], 1))
X_data.shape

(144, 1)

In [193]:
def split_series(sequences, prediction_horizon, look_back):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + look_back
        out_end_ix = end_ix + prediction_horizon - 1
        # check if we are beyond the dataset
        if out_end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix - 1:out_end_ix, -1]

        # Pad sequences if necessary
        if seq_x.shape[0] < look_back:
            seq_x = np.pad(seq_x, ((0, look_back - seq_x.shape[0]), (0, 0)), 'constant')
        if seq_y.shape[0] < prediction_horizon:
            seq_y = np.pad(seq_y, (0, prediction_horizon - seq_y.shape[0]), 'constant')

        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [195]:
print(df.shape)
print(df.head())

(144, 3)
      Date  Quantity  month
0  1949-01       112      1
1  1949-02       118      2
2  1949-03       132      3
3  1949-04       129      4
4  1949-05       121      5


In [200]:
# Convert dataframe to NumPy array
data_array = df[['Quantity', 'month']].values

# Split the data using the split_series function
X_train, y_train = split_series(data_array, prediction_horizon=12, look_back=24)

In [221]:

def rmse(y_true, y_pred):
  return np.sqrt(mean_squared_error(y_true, y_pred))

In [224]:
# Prepare sequences
def create_sequences(df, look_back, n_future):
    X1, y1 = [], []
    for i in range(len(df) - look_back - n_future + 1):
        X1.append(df[i:(i + look_back)])
        y1.append(df[(i + look_back):(i + look_back + n_future)])
    return np.array(X), np.array(y)

look_back = 24
n_future = 7
X1, y1 = create_sequences(data_array, look_back, n_future)

In [225]:
# Split into training and validation sets
X_train1, X_val1, y_train1, y_val1 = train_test_split(X1, y1, test_size=0.2, shuffle=False)

In [230]:
print(X_train1.shape, X_val1.shape)

(537, 1000) (135, 1000)


In [229]:
X_train, X_val1 = split_dataset(df, prediction_horizon,look_back)
print(X_train.shape, X_test.shape)

before split
(132, 3)
(36, 3)



/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (5,) + inhomogeneous part.

In [231]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense
from tensorflow.keras.activations import swish
import tensorflow as tf
# Custom Swish activation function
def swish(x):
    return x* tf.keras.backend.sigmoid(x)

Final_Exam_Starter_Code.ipynb” and is called “split_series”. Using the function for prediction_horizon of 12 and look_back value of 24. Train a stacked GRU with the following specifications: 2 GRU layers with 50 and 25 hidden units and ‘swish’ activation function, rmsprop optimizer with learning_rate 0.001, and loss function of mse, batch size 128 and number of epochs 200.

In [233]:
# Build the model
model1 = Sequential()
model1.add(GRU(50, return_sequences=True, activation=swish, input_shape=(look_back, 1)))
model1.add(GRU(25, activation=swish))
model1.add(Dense(n_future))
model1.compile(optimizer=RMSprop(learning_rate=0.001), loss='mse')


model1.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_24"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru_2 (GRU)                          │ (None, 24, 50)              │           7,950 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_3 (GRU)                          │ (None, 25)                  │           5,775 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_23 (Dense)                     │ (None, 7)                   │             182 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 13,907 (54.32 KB)

 Trainable params: 13,907 (54.32 KB)

 Non-trainable params: 0 (0.00 B)

In [235]:
history1 = model1.fit(X_train1, y_train1, epochs=200, batch_size=128, validation_split=0.2)

Epoch 1/200


ValueError: Dimensions must be equal, but are 5 and 7 for '{{node compile_loss/mse/sub}} = Sub[T=DT_FLOAT](data_1, sequential_24_1/dense_23_1/Add)' with input shapes: [?,5], [?,7].

In [214]:
prediction_horizon = 12
look_back = 24

# Split the data into sequences
X, y = split_series(df, look_back, prediction_horizon)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

InvalidIndexError: (slice(0, 12, None), slice(None, -1, None))

In [215]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense
from tensorflow.keras.optimizers import RMSprop

# Define the model
model = Sequential([
    GRU(50, activation='swish', return_sequences=True, input_shape=(look_back, X_train.shape[2])),
    GRU(25, activation='swish'),
    Dense(prediction_horizon)  # Output layer
])

# Compile the model
model.compile(optimizer=RMSprop(learning_rate=0.001), loss='mse')

# Print model summary
model.summary()

IndexError: tuple index out of range

In [ ]:
# Train the model
history = model.fit(
    X_train, y_train,
    epochs=200,
    batch_size=128,
    validation_split=0.1,  # Using a portion of training data for validation
    verbose=1
)

In [ ]:
# Evaluate the model
test_loss = model.evaluate(X_test, y_test, verbose=1)
print(f'Test Loss: {test_loss}')

In [ ]:
predictions = model.predict(X_test)

i. Calculate the average RMSE over n_test_samples = 2. (Note: RMSE
formula is given).


In [176]:
def rmse(y_pred, y_real):
    return np.sqrt(np.sum((y_pred - y_real)**2)  / len(y_real))